In [ ]:
import pickle

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import random
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
root_dir = '../input/traffic-signs-preprocessed/'

In [ ]:
import pandas as pd
label_names = pd.read_csv("../input/traffic-signs-preprocessed/label_names.csv")

In [ ]:
train = pickle.load(open(root_dir + 'train.pickle', 'rb'))
valid = pickle.load(open(root_dir + 'valid.pickle', 'rb'))
test = pickle.load(open(root_dir + 'test.pickle', 'rb'))
labels = pickle.load(open(root_dir + 'labels.pickle', 'rb'))

In [ ]:
X_train = train['features']
y_train = train['labels']

X_validation = valid['features']
y_validation = valid['labels']

X_test = test['features']
y_test = test['labels']

In [ ]:
X_train.shape

In [ ]:
y_train.shape

## Visualize dataset

In [ ]:
i = 3100
plt.imshow(X_train[i])
y_train[i]

In [ ]:
i = 3100
plt.imshow(X_validation[i])
y_validation[i]

In [ ]:
i = 2100
plt.imshow(X_test[i])
y_test[i]

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
X_train_gray = np.sum(X_train/3, axis =3, keepdims = 3)
X_validation_gray = np.sum(X_validation/3, axis =3, keepdims = 3)
X_test_gray_gray = np.sum(X_test/3, axis =3, keepdims = 3)

In [ ]:
X_train_gray.shape

In [ ]:
X_validation_gray.shape

In [ ]:
X_train_gray_norm = (X_train_gray - 128)/128
X_validation_gray_norm = (X_validation_gray - 128)/128
X_test_gray_norm = (X_test_gray_gray - 128)/128

In [ ]:
X_train_gray_norm

In [ ]:
i = 610
plt.imshow(X_train_gray[i].squeeze(), cmap = 'gray')
plt.figure()
plt.imshow(X_train[i])
plt.figure()
plt.imshow(X_train_gray_norm[i].squeeze(), cmap = 'gray')

In [ ]:
i = 500
plt.imshow(X_validation_gray[i].squeeze(), cmap = 'gray')
plt.figure()
plt.imshow(X_validation[i])
plt.figure()
plt.imshow(X_validation_gray_norm[i].squeeze(), cmap = 'gray')

In [ ]:
i = 500
plt.imshow(X_test_gray_gray [i].squeeze(), cmap = 'gray')
plt.figure()
plt.imshow(X_test[i])
plt.figure()
plt.imshow(X_test_gray_norm[i].squeeze(), cmap = 'gray')

In [ ]:
from tensorflow.keras import datasets,layers,models
LeNet = models.Sequential()
LeNet.add(layers.Conv2D(6, (5,5), activation = 'relu', input_shape = (32, 32, 1)))
LeNet.add(layers.AveragePooling2D(2,2))

LeNet.add(layers.Conv2D(16, (5,5), activation = 'relu'))
LeNet.add(layers.AveragePooling2D(2,2))

LeNet.add(layers.Flatten())

LeNet.add(layers.Dense(120, activation = 'relu'))
LeNet.add(layers.Dense(84, activation = 'relu'))
LeNet.add(layers.Dense(43, activation = 'softmax'))
LeNet.summary()


In [ ]:
LeNet.compile(optimizer = 'Adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = LeNet.fit(X_train_gray_norm, y_train, batch_size = 500, nb_epoch = 50, verbose = 1, validation_data = (X_validation_gray_norm, y_validation))

## Evaluation

In [ ]:
score = LeNet.evaluate(X_test_gray_norm, y_test)
print('Test Accuracy is : {}'.format(score[1]))

In [ ]:
history.history.keys()

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss  = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label = 'Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label = 'val_accuracy')
plt.title("training and validation accuracy")
plt.legend()

In [ ]:
plt.plot(epochs, loss, 'ro', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'val_loss')
plt.title("training and validation loss")
plt.legend()

In [ ]:
predicted_classes = LeNet.predict_classes(X_test_gray_norm)
y_true = y_test

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(predicted_classes, y_true)
plt.figure(figsize = (20, 20))
sns.heatmap(cm, annot = True)

In [ ]:
W_grid = 5
L_grid = 5
fig, axes = plt.subplots(W_grid, L_grid, figsize = (12, 12))
axes = axes.ravel()
for i in np.arange(0, W_grid*L_grid):
    axes[i].imshow(X_test[i])
    axes[i].set_title("Prediction class = {:0.1f}\n True class = {:0.1f}".format(predicted_classes[i], y_true[i]))
    axes[i].axis('off')
plt.subplots_adjust(wspace = 1)